This notebook details the full implementation of KAK on two-qubit (with Weyl chamber)! More background is provided in the other two notebooks (Cartan Decomposition - KAK & Weyl Chamber).

### I. Background

The Lie algebra $\mathfrak{su}(4)$ is given by

$$\mathfrak{su}(4) = \text{span}_\mathbb{R}i \{ \text{IX, IY, IZ}, \mathbf{XI}, \mathbf{XX}, \mathbf{XY}, \mathbf{XZ}, \text{YI, YX, YY, YZ}, \mathbf{ZI}, \mathbf{ZX}, \mathbf{ZY}, \mathbf{ZZ} \}$$

Note that $\text{I}^{\otimes 2}$ is not included because the its exponential is the global phase. 

For reasons that will become clear, we choose type **AI** Cartan involution.

For $u \in \mathfrak{su}(4)$, type **AI** Cartan involution is given by

$$\theta(u) = -u^T$$

with its global Cartan involution being $\Theta(U) = U^\ast$.

Observe the action of $\theta$ on $\mathfrak{k}$ and $\mathfrak{m}$ subspaces.

$$\theta(\mathfrak{k}) = -\mathfrak{k}^T = \mathfrak{k}, \quad \theta(\mathfrak{m}) = -\mathfrak{m}^T = -\mathfrak{m}$$

It follows that $\mathfrak{k}^T = -\mathfrak{k}$ and $\mathfrak{m}^T = \mathfrak{m}$. Thus $\mathfrak{k}$ is skew-symmetric and $\mathfrak{m}$ is symmetric. Then, one can check that

$$\mathfrak{k} = \text{span}_\mathbb{R}i\{\text{IY, XY, ZY}, \mathbf{YI}, \mathbf{YX}, \mathbf{YZ}\}, \quad \mathfrak{m} = \text{span}_\mathbb{R}i\{\text{IX, IY, IZ}, \mathbf{XI}, \mathbf{XX}, \mathbf{XZ}, \text{ZI, ZX, ZZ}, \mathbf{YY}\}$$

We choose our Cartan subalgebra to be $\mathfrak{h} = \text{span}_\mathbb{R}i\{\text{IZ, ZI, ZZ}\}$.

Observe that the dimension of $\mathfrak{k} = SO(4)$ and $SU(2) \otimes SU(2)$ coincides. As such, there is a homomorphism between the two Lie algebra (and Lie group) by the conjugation of the "magic basis"

$$Ad_B (\mathfrak{so}(4)) = \mathfrak{su}(2) \oplus \mathfrak{su}(2)$$

where $B$ is the matrix (up to switching columns)

$$B = \frac{1}{\sqrt2}
\begin{pmatrix} 
1 & 0 & 0 & i \\
0 & i & 1 & 0 \\
0 & i & -1 & 0 \\
1 & 0 & 0 & -i
\end{pmatrix}$$

Importantly, $Ad_B$ maps $\mathfrak{h}$ to another Cartan subalgebra by

$$Ad_B(\text{IZ}) = -\text{YY}, \quad Ad_B(\text{ZI}) = \text{XX}, \quad Ad_B(\text{ZZ}) = \text{ZZ}$$

We can also check the $\mathfrak{k}$-subalgebra homomorphism.

\begin{align}
Ad_B(\text{IY}) = -\text{IX}, \quad Ad_B(\text{XY}) = -\text{ZI}, \quad Ad_B(\text{ZY}) = -\text{XI} \\
Ad_B(\text{YI}) = -\text{YI}, \quad Ad_B(\text{YX}) = -\text{IZ}, \quad Ad_B(\text{YZ}) = \text{IY} \\
\end{align}

### II. Demonstration

In [1]:
# Import
import numpy as np
from scipy.stats import unitary_group
from sympy import Matrix
from scipy.linalg import expm
from typing import Tuple

#### Helper Functions

In [2]:
# Helper Functions

I = np.identity(2)
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

def decompose_one_qubit_product(
    U: np.ndarray, validate_input: bool = True, atol: float = 1e-8, rtol: float = 1e-5
):
    """
    Decompose a 4x4 unitary matrix to two 2x2 unitary matrices.
    Args:
        U (np.ndarray): input 4x4 unitary matrix to decompose.
        validate_input (bool): if check input.
    Returns:
        phase (float): global phase.
        U1 (np.ndarray): decomposed unitary matrix U1.
        U2 (np.ndarray): decomposed unitary matrix U2.
        atol (float): absolute tolerance of loss.
        rtol (float): relative tolerance of loss.
    Raises:
        AssertionError: if the input is not a 4x4 unitary or
        cannot be decomposed.
    """

    """if validate_input:
        assert np.allclose(
            makhlin_invariants(U, atol=atol, rtol=rtol), (1, 0, 3), atol=atol, rtol=rtol
        )"""

    i, j = np.unravel_index(np.argmax(U, axis=None), U.shape)

    def u1_set(i):
        return (1, 3) if i % 2 else (0, 2)

    def u2_set(i):
        return (0, 1) if i < 2 else (2, 3)

    u1 = U[np.ix_(u1_set(i), u1_set(j))]
    u2 = U[np.ix_(u2_set(i), u2_set(j))]

    u1 = to_su(u1)
    u2 = to_su(u2)

    phase = U[i, j] / (u1[i // 2, j // 2] * u2[i % 2, j % 2])

    return phase, u1, u2

def to_su(u: np.ndarray) -> np.ndarray:
    """
    Given a unitary in U(N), return the
    unitary in SU(N).
    Args:
        u (np.ndarray): The unitary in U(N).
    Returns:
        np.ndarray: The unitary in SU(N)
    """

    return u * complex(np.linalg.det(u)) ** (-1 / np.shape(u)[0])

#### Step 0. Generating $U \in SU(4)$

First, we generate a random matrix in $U(4)$.

In [3]:
U = unitary_group.rvs(4, random_state = 18)

Now, we want normalize the determinant of the matrix to make sure $U \in SU(4)$. 

Since $\det(\alpha A) =\alpha^{\dim(A)}\det(A)$, we set $\alpha = \det(A)^{-\frac{1}{\dim(A)}}$ to ensure $\det(\alpha A) = 1$.

In [4]:
U = U / np.linalg.det(U)**0.25 

#### Step 1. Magic Basis Unconjugation

Define the following "magic basis" matrix

To make use of the $SO(4) \to SU(2) \otimes SU(2)$ homomorphism later, let $U^\prime = B^\dagger U B$.

In [5]:
# Magic Basis
B = 1/np.sqrt(2) * np.array([[1, 0, 0, 1j], 
                             [0, 1j, 1, 0], 
                             [0, 1j, -1, 0], 
                             [1, 0, 0, -1j]])

In [6]:
U_prime = np.conj(B).T@U@B

#### Step 2. Isolating the Maximal Torus

Define Type **AI** global Cartan involution

In [7]:
Theta = lambda U: np.conj(U)

By Cartan decomposition, we have $U = KM$, where $K \in \exp(\mathfrak{k})$ and $M \in \exp(\mathfrak{m})$. Observe that 

\begin{align}
\Theta(U^\dagger)U 
&= \Theta(M^\dagger K^\dagger)KM \\
&= M K^\dagger K M \\
&= M^2.
\end{align}

In our case, $\Theta(U^\dagger)U  = U^TU$.

In [8]:
M_squared = Theta(np.conj(U_prime).T)@U_prime

M_squared = np.round(M_squared, 14) # For numerical stability

#### Step 3. Extracting $K_2$

Recall that the maximal torus in $SU(n)$ is isomorphic to $T^{n-1}$. Incidentally, the group $T^{n-1}$ consists of diagonal matrices with determinant $1$, that is 

$$T^{n-1} \cong \exp(\mathfrak{h}) = A$$

Furthermore, by adjoint orbit theorem, $M$ are conjugate of $A$ under some element of $P \in \exp(\mathfrak{k})$. Specifically,

$$M = Ad_P(A)$$

Diagonalizing $M^2$, we get

$$M^2 = PDP^\dagger$$

In [9]:
D, P = np.linalg.eig(M_squared)

Note that $M$ will share the same eigenvectors as $M^2$. This means that $P \in SO(4)$.

**<span style="color:red">Warning:</span>** Sometimes computing the diagonalization of $M^2$ leaves $P$ with determinant of $-1$ instead of $+1$. When this happens, the magic basis homomorphism cannot map $P$ to $SU(2) \otimes SU(2)$. We can fix this by simply **multiplying the first column of $P$ by -1**.

In [10]:
print(f"Determinant of P: {np.round(np.linalg.det(P).real, 14)}")

Determinant of P: -1.0


In [11]:
if np.isclose(np.linalg.det(P), -1):
    P[:, 0]*=-1  # Multiply the first eigenvector by -1

Verify that $P \in SO(4)$

In [12]:
print(f"P is orthogonal: {np.allclose(P.T@P, np.identity(4))}")
print(f"det(P) = 1: {np.isclose(np.linalg.det(P), 1)}")

P is orthogonal: True
det(P) = 1: True


And now we can let $K_2 = P^\dagger$.

In [13]:
K2 = np.conj(P).T

#### Step 4: Extracting $A$

Now, we find that $M = PD^{1/2}P^\dagger$. Recall that all maximal torus in $SU(n)$ are isomorphic to $\mathbb{T}^{n-1}$, which is the group of diagonal matrix with unit determinant. Since we have that $P \in \mathfrak{k} = SO(4)$, if we can ensure that $A = D^{1/2} \in \mathbb{T}^{n-1}$ , then by adjoint orbit theorem, $M = \exp(\mathfrak{m})$ as desired.

In [14]:
A = np.sqrt(D)

**<span style="color:red">Warning:</span>** Similar to above, sometimes taking the square root of $D$ leaves a matrix with determinant $-1$. To fix this, we just need to **multiply the first eigenvalue by $-1$**. Doing so will correct the determinant, but still preserve the condition $A^2 = D$.

In [15]:
print(f"Determinant of A: {np.round(np.prod(A).real, 14)}")

Determinant of A: -1.0


In [16]:
if np.isclose(np.prod(A), -1):
    A[0] *= -1 # Multiply the first eigenvalue by -1
    
A = np.diag(A) # Turn the list of eigenvalues into a diagonal matrix

Verify that $\det(A) = 1$

In [17]:
print(f"det(A) = 1: {np.isclose(np.linalg.det(A), 1)}")

det(A) = 1: True


#### Step 5: Extracting $K_1$

So far, we have $U^\prime = KM = KK_2^\dagger A K_2$. It is easy to see that $K_1 = KK_2^\dagger$. Thus $K_1 = U^\prime K_2^\dagger A^\dagger$. To show that $K_1 \in SO(4)$ check

\begin{align}
K_1^T K_1 
&= U^\prime K_2^\dagger A^\dagger (U^\prime K_2^\dagger A^\dagger)^T \\
&= U^\prime K_2^\dagger A^\dagger A^\ast K_2^\ast (U^\prime)^T \\ 
&= U^\prime K_2^T A^\ast A^\ast K_2 (U^\prime)^T \quad \big( K_2 \in SO(4) \big)\\
&= U^\prime K_2^T (A^\ast)^2 K_2 (U^\prime)^T
\end{align}

Moving both the $U$ terms to the LHS,

\begin{align}
(U^\prime)^\dagger K_1^T K_1 \big((U^\prime)^T\big)^\dagger
&= K_2^T (A^\ast)^2 K_2(U^\prime)^T
\end{align}

Taking the inverse of both sides

\begin{align}
\bigg((U^\prime)^\dagger K_1^T K_1 \big((U^\prime)^T\big)^\dagger \bigg)^\dagger
&= \big(K_2^T (A^\ast)^2 K_2) \big)^\dagger
\end{align}

This is equivalent to

\begin{align}
(U^\prime)^T K_1^T K_1 U^\prime 
&= K_2^T A^2 K_2 \\
&=(U^\prime)^T U^\prime \quad \big( A^2 = D, K_2^\dagger = P \big)
\end{align}

This implies that $K_1^T K_1 = I_4$. Taking the transpose of both sides, we have $K_1 K_1^T = I_4$.

To show that $\det(K_1) = 1$, observe that

\begin{align}
\det(K_1) 
&= \det(U^\prime K_2^\dagger A^\dagger) \\
&= \det(U^\prime) \det(K_2^\dagger) \det(A^\dagger) \\
&= 1
\end{align}

as desired. Thus $K_1 \in SO(4)$.

In [18]:
K1 = U_prime @ np.conj(K2).T @ np.conj(A).T

Verify $K_1 \in SO(4)$.

In [19]:
print(f"K1 is orthogonal: {np.allclose(K1.T@K1, np.identity(4))}")
print(f"det(K1) = 1: {np.isclose(np.linalg.det(K1), 1)}")

K1 is orthogonal: True
det(K1) = 1: True


Now that we have all the pieces, let's verify that 

$$U^\prime = K_1 A K_2 $$

In [20]:
print(f"KAK = U': {np.allclose(U_prime, K1@A@K2)}")

KAK = U': True


#### Step 6: Extracting Local Gates

Recall that $U^\prime = B^\dagger U B$, this means that

\begin{align}
U 
&= B K_1 A K_2 B^\dagger \\
&= (B K_1 B^\dagger) (BAB^\dagger) (B K_2 B^\dagger) \\
\end{align}

Since $K_1, K_2 \in SO(4)$, the magic basis homomorphism ensures that $BK_1 B^\dagger, BK_2 B^\dagger \in SU(2) \otimes SU(2)$. Let $L = BK_1 B^\dagger$ and $R = BK_2 B^\dagger$. Then,

$$L = L_1 \otimes L_2, \quad R = R_1 \otimes R_2$$

In [21]:
L = B@K1@np.conj(B).T # Left Local Product
R = B@K2@np.conj(B).T # Right Local Product

phase1, L1, L2 = decompose_one_qubit_product(L) # L1 (top), L2(bottom)
phase2, R1, R2 = decompose_one_qubit_product(R) # R1 (top), R2(bottom)

Verify correctness

In [22]:
print(f"R Correct: {np.allclose(phase2*np.kron(R1, R2), R)}")

L Correct: True
R Correct: True


#### Step 7: Extracting the Canoncial Parameters

We want to find the canonical parameters $a_0, a_1, a_2$ such that 

$$\exp i(a_0 IZ + a_1 ZI + a_2 ZZ) = A$$

Expanding both sides into matrix form, recall that $A$ is a diagonal matrix and both sides each has unit determinant.

\begin{align}
\begin{pmatrix} 
e^{i(a_0+a_1+a_2)} & 0 & 0 & 0 \\
0 & e^{i(-a_0+a_1-a_2)} & 0 & 0 \\
0 & 0 & e^{i(a_0-a_1-a_2)} & 0 \\
0 & 0 & 0 & e^{i(-a_0-a_1+a_2)} \\
\end{pmatrix} &=
\begin{pmatrix} 
e^{i\theta_0} & 0 & 0 & 0 \\
0 & e^{i\theta_1} & 0 & 0 \\
0 & 0 & e^{i\theta_2} & 0 \\
0 & 0 & 0 & e^{-i(\theta_0 + \theta_1 + \theta_2)} \\
\end{pmatrix}
\end{align}

We have four linear equations

\begin{align}
a_0+a_1+a_2 &= \theta_0 \\
-a_0+a_1-a_2 &= \theta_1 \\
a_0-a_1-a_2 &= \theta_2 \\
-a_0-a_1+a_2 &= -(\theta_0 + \theta_1 + \theta_2)
\end{align}

Notice that the last equation is just the negative of the sum of the first three equations, therefore it is redundant. Hence, we have three equations on three variables. This means that we can set up a matrix equation and solve by taking the inverse.

$$\underbrace{\begin{pmatrix} 
1 & 1 & 1 \\
-1 & 1 & -1 \\
1 & -1 & -1
\end{pmatrix}}_{C} 
\underbrace{\begin{pmatrix}
a_0 \\ a_1 \\ a_2
\end{pmatrix}}_{\vec{a}} = 
\underbrace{\begin{pmatrix}
\theta_0 \\ \theta_1 \\ \theta_2
\end{pmatrix}}_{\vec{\theta}}$$

Then the desired parameters are simply $\vec{a} = C^{-1} \vec{\theta}$. In fact, we give the explicit expression for $C^{-1}$:

$$C^{-1} = \frac{1}{2} \begin{pmatrix} 1 & 0 & 1 \\ 1 & 1 & 0 \\ 0 & -1 & -1 \end{pmatrix}$$

In [23]:
C = np.array([[1, 1, 1], 
              [-1, 1, -1], 
              [1, -1, -1]]) # Coefficient Matrix

In [24]:
theta_vec = np.angle(np.diag(A))[:3] # theta vector
a0, a1, a2 = np.linalg.inv(C)@theta_vec # Computing the "a"-vector

Verify that $\exp i(a_0 IZ + a_1 ZI + a_2 ZZ) = A$

In [25]:
print(
    f"e^i(a.h)=A: {np.allclose(A, expm(1j*(a0*np.kron(I, Z) + a1*np.kron(Z, I) + a2*np.kron(Z, Z))))}"
)

e^i(a.h)=A: True


Recall that $Ad_B$ maps $\mathfrak{h}$ to another Cartan subalgebra by

$$Ad_B(\text{IZ}) = -\text{YY}, \quad Ad_B(\text{ZI}) = \text{XX}, \quad Ad_B(\text{ZZ}) = \text{ZZ}$$

Thus, the canonical parameters of this Cartan subalgebra is given by

$$c_{0} = 2a_1, \quad c_{1} = -2a_0, \quad c_{2} = 2a_2$$

which means that

$$\exp \frac{i}{2}(c_0 XX + c_1 YY + c_2 ZZ) = B A B^\dagger$$

We call the $\text{CAN}(c_0, c_1, c_2)$ gate.

In [26]:
# Define the Canonical gate
CAN = lambda c0, c1, c2: expm(1j/2*(c0*np.kron(X, X) + c1*np.kron(Y, Y) + c2*np.kron(Z, Z)))

In [27]:
c0, c1, c2 = 2*a1, -2*a0, 2*a2 # Unpack parameters

Verify that $\text{CAN}(c_0, c_1, c_2) = BAB^\dagger$

In [35]:
print(f"CAN = BAB^†: {np.allclose(B@A@np.conj(B).T, CAN(c0, c1, c2))}")

CAN = BAB^†: True


Putting everything together, we have

$$U = (L_1 \otimes L_2) \ \text{CAN}(c_0, c_1, c_2) \ (R_1 \otimes R_2)$$

Verify this

In [36]:
print(f"U = L.CAN.R: {np.allclose(U, (phase1*np.kron(L1, L2))@CAN(c0, c1, c2)@(phase2*np.kron(R1, R2)))}")

U = L.CAN.R: True


### III. Method

In [144]:
def weyl_chamber(c):
    """Bring coordinates vector into the Weyl chamber"""

    # Step 0: work in terms of multiple of pi
    c /= np.pi

    # Step 1: Bring everything into [0, 1)
    c -= np.floor(c)

    # Step 2: Sort c1 >= c2 >= c3
    c = np.sort(c)[::-1]

    # Step 3: if c1 + c2 >= 1, transform (c1, c2, c3) -> (1-c2, 1-c1, c3)
    if c[0]+c[1] >=1:
        c = np.sort(np.array([1-c[1], 1-c[0], c[2]]))[::-1]

    # Step 4: if c3 = 0 and c1>1/2, transform (c1, c2, 0) -> (1-c1, c2, 0)
    if (c[0] > 1/2) and np.isclose(c[2], 0):
        c = np.array([1-c[0], c[1], 0])

    # Step 5: Turn it back into radians
    c *= np.pi
    
    return c

def KAK_2q(
    U: np.ndarray,
    rounding: int = 19
) -> Tuple[float, np.ndarray, np.ndarray, float, np.ndarray, np.ndarray, float,
           float, float]:
    """
    Decomposes a 2-qubit unitary matrix into the product of three matrices:
    KAK = L @ CAN(theta_vec) @ R where L and R are two-qubit local unitaries, 
    CAN is a 3-parameter canonical matrix, and theta_vec is a vector of 3 angles.

    Args:
        U (np.ndarray): 2-qubit unitary matrix
        rounding (int): Number of decimal places to round intermediate 
        matrices to (default 14)

    Returns:
        Tuple of 9 values:
            - phase1 (float): Global phase factor for left local unitary L
            - L1 (np.ndarray): Top 2x2 matrix of left local unitary L
            - L2 (np.ndarray): Bottom 2x2 matrix of left local unitary L
            - phase2 (float): Global phase factor for right local unitary R
            - R1 (np.ndarray): Top 2x2 matrix of right local unitary R
            - R2 (np.ndarray): Bottom 2x2 matrix of right local unitary R
            - c0 (float): XX canonical parameter in the Weyl chamber
            - c1 (float): YY canonical parameter in the Weyl chamber
            - c2 (float): ZZ canonical parameter in the Weyl chamber
    """

    # 0. Map U(4) to SU(4) (and phase)
    global_phase = np.linalg.det(U)**0.25
    U /= global_phase

    assert np.isclose(np.linalg.det(U), 1), "Determinant of U is not 1"

    # 1. Unconjugate U into the magic basis
    B = 1 / np.sqrt(2) * np.array([[1, 0, 0, 1j], [0, 1j, 1, 0],
                                   [0, 1j, -1, 0], [1, 0, 0, -1j]]) # Magic Basis
    U_prime = np.conj(B).T @ U @ B

    # Isolating the maximal torus
    M_squared = Theta(np.conj(U_prime).T) @ U_prime

    if rounding is not None:
        M_squared = np.round(M_squared, rounding)  # For numerical stability

    ## 2. Diagonalizing M^2
    D, P = np.linalg.eig(M_squared)

    ## Check and correct for det(P) = -1
    if np.isclose(np.linalg.det(P), -1):
        P[:, 0] *= -1  # Multiply the first eigenvector by -1

    # 3. Extracting K2
    K2 = np.conj(P).T

    assert np.allclose(K2 @ K2.T, np.identity(4)), "K2 is not orthogonal"
    assert np.isclose(np.linalg.det(K2), 1), "Determinant of K2 is not 1"

    # 4. Extracting A
    A = np.sqrt(D)

    ## Check and correct for det(A) = -1
    if np.isclose(np.prod(A), -1):
        A[0] *= -1  # Multiply the first eigenvalue by -1

    A = np.diag(A)  # Turn the list of eigenvalues into a diagonal matrix

    assert np.isclose(np.linalg.det(A), 1), "Determinant of A is not 1"

    # 5. Extracting K1
    K1 = U_prime @ np.conj(K2).T @ np.conj(A).T

    assert np.allclose(K1 @ K1.T, np.identity(4)), "K1 is not orthogonal"
    assert np.isclose(np.linalg.det(K1), 1), "Determinant of K1 is not 1"

    # 6. Extracting Local Gates
    L = B @ K1 @ np.conj(B).T  # Left Local Product
    R = B @ K2 @ np.conj(B).T  # Right Local Product

    phase1, L1, L2 = decompose_one_qubit_product(L)  # L1 (top), L2(bottom)
    phase2, R1, R2 = decompose_one_qubit_product(R)  # R1 (top), R2(bottom)

    # 7. Extracting the Canonical Parameters
    C = np.array([[1, 1, 1], [-1, 1, -1], [1, -1, -1]])  # Coefficient Matrix

    theta_vec = np.angle(np.diag(A))[:3]  # theta vector
    a0, a1, a2 = np.linalg.inv(C) @ theta_vec  # Computing the "a"-vector

    # 8. Unpack Parameters and Put into Weyl chamber
    c0, c1, c2 = weyl_chamber(np.array([2 * a1, -2 * a0, 2 * a2])) 
    
    assert np.allclose(U, (phase1 * np.kron(L1, L2)) @ CAN(c0, c1, c2)
                       @ (phase2 * np.kron(R1, R2))), "U does not equal KAK"

    return phase1, L1, L2, phase2, R1, R2, c0, c1, c2

In [5]:
from sympy import *

In [6]:
theta, phi_0, phi_1 = symbols('theta, phi0, phi1')


In [43]:
aams = Matrix(
    [
        [cos(theta / 2), 0, 0, -I * exp(-I * (phi_0 + phi_1)) * sin(theta / 2)],
        [0, cos(theta / 2), -I * exp(-I * (phi_0 - phi_1)) * sin(theta / 2), 0],
        [0, -I * exp(I * (phi_0 - phi_1)) * sin(theta / 2), cos(theta / 2), 0],
        [-I * exp(I * (phi_0 + phi_1)) * sin(theta / 2), 0, 0, cos(theta / 2)],
    ]
)

In [89]:
ms = aams_matrix.subs({"theta": pi / 2})
xx = aams_matrix.subs([(phi_0, 0), (phi_1, 0)])

In [90]:
xx

Matrix([
[   cos(theta/2),               0,               0, -I*sin(theta/2)],
[              0,    cos(theta/2), -I*sin(theta/2),               0],
[              0, -I*sin(theta/2),    cos(theta/2),               0],
[-I*sin(theta/2),               0,               0,    cos(theta/2)]])

In [91]:
B = 1/sqrt(2) * Matrix([[1, 0, 0, I], [0, I, 1, 0], [0, I, -1, 0], [1, 0, 0, -I]])

In [92]:
U_prime = B.conjugate().T@xx@B

In [93]:
M_squared = simplify(U_prime.T@U_prime)

In [99]:
M_squared.subs(theta, pi/2).evalf()

Matrix([
[-1.0*I,      0,     0,     0],
[     0, -1.0*I,     0,     0],
[     0,      0, 1.0*I,     0],
[     0,      0,     0, 1.0*I]])

In [94]:
P, D = M_squared.diagonalize()

MatrixError: Matrix is not diagonalizable

In [ ]:
simplify(D)

In [53]:
lam = symbols('lambda')

In [79]:
simplify(M_squared)

Matrix([
[-I*exp(I*(phi0 + phi1))*sin(theta)/2 + cos(theta) - I*exp(-I*(phi0 + phi1))*sin(theta)/2,                                                                                        0,                                                                                       0,                                                           I*sin(theta)*sin(phi0 + phi1)],
[                                                                                       0, -I*exp(I*(phi0 - phi1))*sin(theta)/2 + cos(theta) - I*exp(-I*(phi0 - phi1))*sin(theta)/2,                     -(exp(2*I*phi0) - exp(2*I*phi1))*exp(-I*(phi0 + phi1))*sin(theta)/2,                                                                                       0],
[                                                                                       0,                      -(exp(2*I*phi0) - exp(2*I*phi1))*exp(-I*(phi0 + phi1))*sin(theta)/2, I*exp(I*(-phi0 + phi1))*sin(theta)/2 + I*exp(I*(phi0 - phi1))*sin(theta)/2 + cos(theta),  

In [85]:
simplify(det(M_squared - lam*eye(4)))

KeyboardInterrupt: 

In [101]:
import numpy as np

In [102]:
X12 = np.array([[0, 1], [0, 0]])

In [ ]:
X12

In [ ]:
P

In [ ]:
B.conjugate